# DoS攻击流量分析

## DoS 攻击概述

DoS是Denial of Service 的简称，即拒绝服务，指网络信息系统不能正常地向用户提供服务。

造成DoS的攻击行为被称为DoS攻击，是危害系统可用性的一种终极攻击手段,特别是分布式DoS攻击（DDoS attack）。

分布式拒绝服务（Distributed Denial of Service，简称DDoS），指借助于客户/服务器技术，将多个计算机联合起来作为攻击平台，对一个或多个目标发动DDoS攻击，从而成倍地提高拒绝服务攻击的威力。

![DoS攻击](images/14/ddos-attack-infographic.png)

DoS攻击造成的影响和症状有：

- 网络资源耗尽
  + 网络拥塞，网络带宽耗尽
  + 网速奇慢
  
- 计算机资源耗尽
  + 服务器无法连接（TCP资源耗尽)
  - 磁盘被占满
  - CPU负载极高
  - 内存被占满

- 合法用户无法使用信息系统
  + 无法访问网页
  + 无法接收消息
  + 无法连接互联网络
  + 大量的垃圾邮件或垃圾消息

## 常见DoS攻击类型

### 链路层DoS攻击

数据链路层的拒绝服务攻击受协议本身限制，只能发生在局域网内部，这种类型的攻击比较少见。

ARP欺骗或投毒攻击可以算是一种类DoS攻击，它造成了局域网内通信的中断。

![arp投毒原理](images/02/arp投毒原理.png)

### 网络层DoS攻击

网络层（IP层）的DoS攻击通常有两类：

- IP畸形报文攻击
- ICMP泛洪攻击

IP畸形报文攻击，是攻击者向目标主机发送大量的**格式错误的网络数据包，使目标系统在处理这样的数据包时出现崩溃**，从而达到拒绝服务的攻击目的。

下图是IP协议中IP数据包正常分片和重组的示意图。注意：length指数据包长度、ID表示数据包分片ID、Fragflag表示是否还有后续分片、offset表示重组时该分片相对于原数据包的偏移位置。

![ipfragmentation](images/14/ipfragmentation.jpeg)

常见的子类型有：

#### IP畸形包攻击

畸形的IP数据包，会使操作系统协议栈无法处理，最终造成报错、崩溃或拒绝服务。有时也称之为teardrop攻击。

- **IP片段重叠（IP fragment overlapped）**
  
即有两个或多个数据包中，IP分片的offset是重叠的。某些操作系统无法正确处理以这种方式重叠的片段，并可能抛出异常或以其他不安全方式进行处理。

可用于试图绕过入侵检测系统。此时，攻击的一部分与其他随机数据一起以片段形式发送，后续的碎片可能会覆盖先到的随机数据。有些IDS对没有正确地重新组装完成的数据包不做处理，这样的攻击也将不会被检测到。

- **IP分片缓冲区溢出（IP fragment buffer overrun）**

过量接收不完整的IP分片可能会造成缓冲区溢出。这会造成系统崩溃或绕过IDS检查。

- **IP分片溢出（IP fragment overun）**

重组的分片数据包超过声明的IP数据长度，或大于MTU。根据RFC定义，任何IP数据包都不应该大65536字节，超出可能会使系统崩溃。

- **IP分片太多（IP fragment too many packets）**

系统接收到过量的不完整分片数据包，使得系统报错或崩溃。

- **IP分片不完整（IP fragment incomplete packet）**

造成缺少一部分分片造成无法重组原数据包，可鞥会造成系统报错或崩溃。

- **IP分片过小（IP Fragment Too Small）**

任何小于400字节的都可认为是小数据包，这种情况可被认为是故意为之，用于逃避IDS检查。

- **IP头标志设置相互矛盾**

> 其它类型的畸形报文攻击，都利用了“破碎的数据包会使对方操作系统的协议栈瘫痪”这一脆弱性,或传输大于MTU的数据包（大于1500字节）、或发送无法重组的乱序分片（针对TCP / IP重组机制），目标服务器为了处理这些错误，资源很快被消耗，导数服务器OS网络协议栈不可用。。


##### 本节实验操作

- 打开ip_frag_source.pcapng样本文件（这是一个正常的样本）
- 分析第1、第2、第3分组的IP协议头部，完成以下表格：

|分组序号|IPv4 Identification|IP Flag More fragment|Fragment offset|IP包数据（Data）长度|
|-|-|-|-|-|
|1|
|2|
|3|

- 能否从表格中发现offset与data 长度的关系？


- 打开ip_fragment_overlapper.pcap样本文件（这是一个疑似异常的样本）
- 从第2个分组开始，完成下列表格：

|分组序号|IPv4 Identification|IP Flag More fragment|Fragment offset|IP包数据（Data）长度|
|-|-|-|-|-|
|2|0x00f2|0|2015|40|
|3|
|4|
|5|
|6|
|7|


- 根据上述表格中的分析结果，回答以下问题：
- 是否存在IP畸形包？
- 存在哪类IP畸形？判断依据是什么？
- 这种类型的IP数据包被操作系统接收、处理时，可能会造成何种影响？

  
Teardrop攻击，就是一类IP畸形包攻击。

攻击者通过向目标网络或主机发送重叠的、过大的畸形IP数据包，造成对方系统协议栈崩溃或拒绝服务的行为。

Teardrop攻击对Windows 3.1/95/NT都很有效；在Windows 7和vista中也存在类似缺陷。

##### 本节实验操作

- 打开 teardrop.pcap 样本文件（这是一个疑似异常的样本）
- 分析第8和第9分组，完成下列表格：

|分组序号|IPv4 Identification|IP Flag Don't fragment|Fragment offset|IP包长度|
|-|-|-|-|-|
|8|
|9|

- 根据上述表格中的分析结果，回答以下问题：
- 是否存在IP畸形包？
- 存在哪类IP畸形？判断依据是什么？



#### ICMP泛洪攻击

也称Smurf攻击，攻击者通过发送大量的ICMP虚假请求，欺骗网络内的大量主机产生ICMP应答，大量的请求与应答将使网络负载呈几何级数增长。

这类攻击形式较为简单，在网络流量的表现为：

- 大量随机IP地址
- 大量ICMP ECHO 请求
- 大量ICMP 响应



### 传输层DoS攻击

传输层DDoS攻击主要是指TCP 连接类攻击和UDP泛洪攻击。

#### TCP 连接类攻击

- SYN Flood攻击
- ACK Flood攻击
- RST Flood攻击
- TCP 畸形包攻击

以Syn Flood攻击为例，它利用了TCP协议的三次握手机制。当服务端接收到一个Syn请求时，服务端必须使用一个监听队列将该连接保存一定时间。

![tcp连接示意](images/14/tcp连接示意.jpg)

**攻击者通过向服务端不停发送请求，但不应答服务器响应，从而消耗服务端的资源，最终使服务器负载过高乃至崩溃。**

当等待队列被占满时，服务端将无法响应正常用户的请求，达到拒绝服务攻击的目的。

#### UDP 泛洪攻击

UDP泛洪（UDP flood）的实现原理与ICMP泛洪类似，攻击者通过向目标主机发送大量的UDP报文，导致目标主机忙于处理这些UDP报文，而无法处理正常的报文请求或响应。

常见类型：

- UDP Flood攻击
- UDP 多播泛洪攻击
- UDP 畸形包攻击

> UDP多播发送时，必须设置TTL（解决“数据包传递距离”的主要因素），TTL的值设置过大将影响网络流量，当然设置过小也会导致数据无法传递到目的端。

> 缺省情况下，发送 IP 多播数据报时其生存时间TTL = 1。
> - 0 : 限定在同一主机；1 : 限定在同一子网；
> - 32 : 限定在同一站点
> - 64 : 限定在同一地区
> - 128 : 限定在同一洲
> - 255 : 范围不受限制

#### 本节实验操作

- 打开样本文件 tcp_syn_port_scan.pcap文件
- 利用统计工具分析流量图，流类型设为TCP 流。
- 回答以下问题：
  + 出现次数最多的目标主机IP为？
  + 这个IP的多少个端口被尝试连接？
  + 连接情况如何？
  + 通过以上分析，样本记录的网络行为属于何种网络行为？
  
- 打开样本文件 udp-multicast-flood.pcap，回答下列问题：
- 样本中的目的IP地址是什么？
- 这一的IP地址属于何种类型？
- 
  
### 应用层DDoS攻击

Web应用层攻击主要是指HTTP Get Flood，HTTP Post Flood，CC攻击等。

#### HTTP Get Flood 攻击

攻击者利用各种代理向服务器发送大量HTTP Get请求，消耗服务器资源，造成可用性降低的攻击。

![HTTPGet](images/14/HTTPGet.png)

![http-flood-attack](images/14/http-flood-attack.webp)

#### HTTP Post Flood 攻击

攻击者利用各种代理向服务器发送大量HTTP Get请求，消耗服务器资源，造成可用性降低的攻击。

#### CC攻击

Challenge Collapsar 攻击，简称CC攻击。

CC攻击的特点：

- **更加隐蔽**

通常应用层攻击完全模拟用户请求，类似于各种搜索引擎和爬虫一样，这些攻击行为和正常的业务并没有严格的边界，难以辨别。

- **更加严重**

此类攻击的影响**Web服务中一些资源消耗较大的事务和页面，大量消耗服务器资源。**

例如，Web应用中的分页和分表，频繁的翻页将会占用较多的Web服务资源；尤其在高并发频繁调用的情况下，类似这样的事务就成了早期CC攻击的目标。

- **更加难防**

CC攻击瞄准的是Web应用的后端业务，除了导致拒绝服务外，还会直接影响Web应用的功能和性能，包括Web响应时间、数据库服务、磁盘读写等，都会导致功能和性能的异常。

例如，春运期间，各种刷票软件对12306网站的访问，从某种程度上来说就是CC攻击。


#### DNS DDoS攻击

DNS DDoS攻击主要是指DNS Request Flood、DNS Response Flood、虚假源+真实源DNS Query Flood、权威服务器和Local服务器攻击。

以DNS Query Flood攻击为例，其本质上执行的是真实的Query请求，属于正常业务行为。但如果多台肉鸡同时发起海量的域名查询请求，对服务端来说则无法为正常的Query请求返回结果，从而导致拒绝服务。

#### 本节实验操作

- 打开样本文件 udp_dns_flood.pcap，回答以下问题：
- 对样本进行协议分级统计，什么协议类型的数据包是这个样本中的主体？
- 样本中的大量存在的网络行为是什么？
- 这种行为正常么？可能是什么类型的网络事件？

- 打开i

## 案例1：某部网站遭受DDoS攻击的过程分析

网站出现不能正常访问的原因是多种多样的，问题可能发生在哪里呢？

根据经验，一般有以下几种可能：

- 网站服务其端
- 互联网出口
- 网络管理限制
- 网站受到攻击

其中，网站收到攻击，导致无法访问的原因和预防手段是最为复杂和令人难以判断的。

攻击者利用网站漏洞或特定攻击手法，往往隐蔽难以察觉，这就需要网络安全人员：

- 针对特定事件进行分析
- 做到快速响应定位根源
- 迅速恢复网站的访问

下面我们来看一个真实的案例：

### 问题描述
 
某日上午，某部官网突然出现不能访问的情况，维护人员于12点20分左右重启网站应用系统，网站访问恢复正常。但在几分钟后又重复出现不能访问的情况。

从流量上观察，并未发现流量异常突发的情况，具体问题需要进一步分析。

事发期间的流量(横轴为时间，纵轴为网络速率)，如下图所示：

![dos攻击案例图1](images/14/dos攻击案例图1.png)

### 分析过程

#### 对12点20分左右出现的异常现象进行分析

可疑互联网IP X.X.1.132在2分30秒左右内，请求下载了13.rar、14.rar、15.rar等文件共计273次，这些文件的大小均在200MB左右。

从12点18分16秒第一次请求开始，到12点19分35秒服务器出现无响应的情况；

再到12点20分13秒服务器出现HTTP 502报错；

本次攻击只花费不到2分钟的时间就达到了拒绝服务的攻击效果。

![dos攻击案例图2](images/14/dos攻击案例图2.png)

查看X.X.1.132与网站的TCP会话，发现这些会话的持续时间都很长，因为发现攻击后，服务器在两分钟后进行了重启，所以看似持续时间只有2分多钟，但如果不重启会话会一直持续。

![dos攻击案例图3](images/14/dos攻击案例图3.png)

深入分析这些会话内容，可以看到从传输开始，客户端宣告的TCP窗口大小为64860。

![dos攻击案例图4](images/14/dos攻击案例图4.png)

在客户端接收了一些数据后，在该会话的第57个包时，客户端宣告的TCP窗口大小减小到17316。

![dos攻击案例图5](images/14/dos攻击案例图5.png)

到此会话第66个包时，客户端宣告的TCP窗口为756。

![dos攻击案例图6](images/14/dos攻击案例图6.png)

直到第68个包时，客户端宣告的TCP窗口变为0，说明客户端已经不能再接收任何数据。随后服务器将需要继续传输的数据放在发送缓存中，等待客户端窗口恢复正常后再继续发送。

![dos攻击案例图7](images/14/dos攻击案例图7.png)

网站服务器不断的发送ACK数据包去进行TCP窗口探测，每次探测时间为双倍的探测计时器时间，但每次探测的结果均为客户端TCP窗口为0，如下图所示。

![dos攻击案例图8](images/14/dos攻击案例图8.png)

#### 对14点30分左右出现的异常现象进行分析

该部网站在14点30分又受到来自X.X.23.250发起的攻击，攻击方式与12点20分发生的攻击行为一致，最后看到探测器时间一直保持在120秒探测一次，而会话的时间达到了十几分钟甚至更长。

![dos攻击案例图9](images/14/dos攻击案例图9.png)

### 分析结论
 
- 攻击者利用官方网站现有的大文件，向服务器发送下载请求；

- 在传输过程中利用TCP零窗口的特点，只接收少量文件后便不再继续接受文件；

- 导致大文件一直积压在服务器的发送缓存中，造成服务性能消耗，进而造成官网不能访问的现象。

### 防护建议

- 在负载均衡设备上设置TCP零窗口超时时间；
- 将大文件单独放在另外一台服务器中，将官网链接指向到此文件服务器。

### 给我们的启示 

网站正常运行涉及的内容十分繁杂，当遭受黑客的攻击后，维护人员往往很难做到快速响应，使网站恢复正常访问，更难以确定对方是利用的何种手段或漏洞发起的攻击。

不通过网络流量分析方法，很难实现对网络攻击的判断和可视化，很难精准监控网络数据流向，很难尽快将攻击行为梳理出来，实现网络安全快速响应。

## 案例2 发现并防范隐蔽的CC攻击

CC攻击（Challenge Collapsar）类属于DDoS攻击（分布式拒绝服务）。

攻击者借助代理服务器或者肉鸡，生成指向受害主机的合法请求，向受害主机不停发送大量数据，从而造成网络链路拥塞，致使服务器资源耗尽直至宕机崩溃。

本案例详细分析了CC攻击的手段及原理，对我们有效防范CC攻击有启发意义。

### 问题描述
 
科来网络分析工程师应邀为某集团用户进行网络安全分析服务，在通过科来设备检查该集团官网的应用质量时发现异常：

- 在以“1分钟”为时间周期下发现该集团官网应用流量呈现规律性突发状况，如下图所示：

![cc攻击案例图1](images/14/cc攻击案例图1.png)

### 分析过程

为了解具体情况，选取其中的周期性突发流量进行比对，并选取“1秒”刻度展示流量走向，发现每隔5分钟便有持续10余秒的突发流量，流量峰值达到150Mbps，已超过出口链路的带宽，如下图所示。


![cc攻击案例图2](images/14/cc攻击案例图2.png)
![cc攻击案例图3](images/14/cc攻击案例图3.png)

任选多个高发流量分别进行深度分析：发现访问的客户端来自国内外多个地区，客户端总数平均在170个。

在对流量指标进行排序后，发现所有客户端的访问流量大小、数据包数量及其他指标均相似。

由于正常情况下，客户端访问官网产生的流量应该相差较大，故怀疑官网遭受攻击或被扫描。

![cc攻击案例图4](images/14/cc攻击案例图4.png)

针对客户端的访问流量进行深度解码分析，发现该现象具有以下特征：

- 每个客户端对官网服务器（X.X.19.17）访问均达到30条。
  + 那么，按照平均170个客户端数量推算，共计发生5100次并发请求。

- 每个访问请求均是GET合法请求，服务器无法拒绝（见下图中服务器应答为“HTTP/1.1 200 OK”）。
  + 且请求的URL均属于集团及子公司的合法网站域名。每个客户端对这些域名重复性发起请求。

正常访问在打开网站时会附带对图片、插件、控件等元素的请求，下图为正常访问：通过浏览器打开网站时会请求相关图片、控件等元素。

![cc攻击案例图5](images/14/cc攻击案例图5.png)

而本次攻击的请求页面均只访问网站的根路径（见图中URL），如下两图所示。

![cc攻击案例图6](images/14/cc攻击案例图6.png)

下图为正常访问：通过浏览器打开网站时会请求相关图片、控件等元素。

![cc攻击案例图7](images/14/cc攻击案例图7.png)

对其中某一会话进行分析均可见类似“放大攻击”现象。如下图示：客户端请求数值大小为372B，而官网服务器回应数值大小为8.37KB，两者字节数比例1：23.04，相差悬殊。

![cc攻击案例图8](images/14/cc攻击案例图8.png)

### 分析结论

综上分析，可判定官网每隔5分钟遭受一次CC攻击（即HTTP Flood攻击）。

攻击者利用百位数级别的客户端（肉鸡或代理设备）同时向集团及子公司的网站发动“不能被拒绝”的请求（请求的路径均是网站域名的“/”路径）.

由于请求开销远小于官网服务器应答的开销，因此造成“放大攻击”。

### 造成的危害

- 出口链路拥塞：攻击时流量高达150Mbps，超过带宽上限。

- 服务器性能严重消耗：遭受攻击时主机负载率快速提高接近100%，短时间内服务器性能受到严重消耗。

倘若攻击时间持续加长，服务器资源将耗尽并宕机崩溃，如下方服务器性能监控图示。

![cc攻击案例图9](images/14/cc攻击案例图9.png)

- 浏览官网的体验较差。

本例攻击间隔5分钟发动一次，每次攻击只持续10余秒，因单次攻击时长比较短，故官网页面打开缓慢的现象并不明显。

假如CC攻击改为持续性攻击策略，势必会造成出口链路严重拥塞，服务器处理性能枯竭并导致官网访问异常。

### 防护建议

通常发动CC攻击时，攻击者会用攻击软件同时模拟多个用户，向目标网站发起多个请求。

为防止攻击地址被屏蔽，这些软件会利用内置代理攻击的功能，通过多个代理服务器模拟多用户向目标发起攻击，使封锁指定IP的防御方式失效。

本案例中的攻击IP数量庞大且分散无明显规律，常规上以边界防护设备实现阻断IP黑名单的方式难度较大。

由于通过代理发起的攻击方式普遍存在共同特征，找到该特征便可对安全设备自定义特征进行防护。

比对同个客户端不同会话的请求及不同客户端的请求进行分析发现，其http首部都包含同个特征。如下图比对：

Accept-Language:zh-CN,zh;q=0.8,en;q=0.6,it;q=0.4,zh-TW;q=0.2,ja;q=0.2

![cc攻击案例图10](images/14/cc攻击案例图10.png)
![cc攻击案例图11](images/14/cc攻击案例图11.png)
![cc攻击案例图12](images/14/cc攻击案例图12.png)

科来网络分析工程师建议在安全设备WAF上自定义特征过滤，以此防范CC攻击。通过管理员后期反馈了解到：该措施效果明显，服务器性能负载一直处于良好状态。

### 防护效果

WAF自定义防护后，通过视图可见防护效果显著，如下图所示。

![cc攻击案例图13](images/14/cc攻击案例图13.png)

查看WAF外侧采集点捕获的流量交易日志，可见CC攻击的请求服务器均无应答。这表明恶性请求已被WAF成功拦截，请求不再转发到官网服务器，如下图所示。

![cc攻击案例图14](images/14/cc攻击案例图14.png)

对CC攻击的TCP交易进行深度分析：观测到攻击IP发送GET请求，随之WAF命中特征阻断策略并发送FIN报文关闭交易。该策略成功减少了应答数据占用的网络带宽开销及服务器性能的消耗，如下图所示。

![cc攻击案例图15](images/14/cc攻击案例图15.png)

经过上述分析，可以看到恶意请求均被成功拦截，网络流量也得到有效控制。

### 更多的思考

攻击者倘若加大攻击力度，比如增加攻击客户端数、增加请求数、不间断请求等手段，依然会影响到网络下行带宽开销并严重消耗WAF性能，从而影响官网访问的体验。

面对这种情况，如果安全设备可以对同一个攻击IP命中5次（数值可自由设定）特征时，便自动地将该IP标记为黑名单进行封锁（通过设置封锁时间后自动回复，减少误封锁），便可以实现让攻击IP无法与WAF完成三次握手，实现更有效的防护。

## 案例3 发现利用DNS放大攻击服务器的行为

很多网络服务异常，往往是被攻击造成的。由于其产生原因有很多种，如何对异常现象进行分析定位，是解决此类问题的关键。DNS放大攻击是一种拒绝服务攻击，攻击者将僵尸网络中大量的被控主机伪装成被攻击主机，在特定时间点，连续向多个允许递归查询的DNS服务器，发送大量DNS服务请求，迫使其提供应答服务。经DNS服务器放大后的大量应答数据，再发送到被攻击主机，形成攻击流量，导致其无法提供正常服务甚至瘫痪。

### 问题描述
 

X.X.29.4为某大型行业用户的DNS服务器，需要对外提供DNS服务。近期，该用户网络拥塞，运维部门在部署科来网络回溯分析系统后发现，在可疑域名警报功能中触发有大量警报。

![DNS放大攻击案例图1](images/14/DNS放大攻击案例图1.png) 

### 分析过程
 
X.X.157.245（经查为美国IP）在短时间内向X.X.29.4服务器发送了大量的DNS请求，请求的域名为dnsamplicationattacks.cc（DNS Amplification Attacks字面意思就是DNS放大攻击），如下图所示。

![DNS放大攻击案例图2](images/14/DNS放大攻击案例图2.png) 

X.X.157.245发出的请求包为101字节，DNS服务器返回的应答包为445字节，从而使通信流量被放大了4.4倍，如下图所示。

![DNS放大攻击案例图3](images/14/DNS放大攻击案例图3.png) 

攻击者利用大量被控主机，向大量的DNS服务器发送DNS请求。但请求中的源IP地址，均被伪造成被攻击者的IP（在本例中为X.X.157.245），于是DNS服务器会向被攻击者返回查询结果。通常查询应答包会比查询请求包大数倍甚至数十倍（在本例中为4.4倍），从而形成对X.X.157.245地址的流量放大攻击。

在本例中，客户的DNS服务器被作为实施这种DNS放大攻击的代理参与其中，攻击过程见下图。

![DNS放大攻击案例图4](images/14/DNS放大攻击案例图4.png) 

### 分析结论
 
攻击者利用大量被控主机，在短时间内向DNS服务器（X.X.29.4）发送大量的DNS请求，查询应答包会比查询请求包大4.4倍，造成大流量发送到伪造的源IP地址（X.X.157.245），形成对该IP地址的拒绝服务攻击。

### 防护建议

- 增大链路带宽；
- DNS服务器关闭递归查询；
- 一旦发生大规模DNS放大攻击，可以马上与ISP联系，在上游对攻击进行过滤。
 
### 启示

黑客常常利用DNS服务器的特性，将其做为攻击放大器，在保证自身隐蔽性的同时，依靠僵尸网络发布攻击，往往可以制造极大的攻击流量。然而，在本案例中我们看到，通过网络分析技术可以把攻击行为完全梳理出来，达到网络攻击可视化的效果；并且通过协议解码，可以清晰的分析出攻击者使用的攻击手段。利用网络分析技术，通过2到7层的深度协议分析，能够精确解码、详细统计数据，快速发现安全事件，形成应急响应力量，为信息系统以及网络安全保驾护航。

## 案例4 找出对内网的UDP攻击行为

UDP Flood是流量型攻击。由于UDP协议是一种无连接的服务，在UDP Flood攻击中，攻击者可发送大量伪造源IP地址的小UDP包。UDP协议与TCP协议不同，是无连接状态的协议，并且UDP应用协议繁多且彼此差异大，因此针对UDP Flood的防护比较困难。一旦发生UDP攻击，轻则导致网络访问缓慢，重则导致网络瘫痪，不但影响工作，还将为企事业单位带来巨大损失。那么，在网络分析技术下，如何快速解决这类安全问题呢？下面的案例为我们提供了一种清晰的防范思路。

### 问题描述
 
某集团下属单位A矿报告说，该区域网络内有许多用户访问集团公司内部网和互联网缓慢或者不通；下属单位B矿报告说，到机关总部网络故障。

针对此类报告，对相关网络进行排查，发现有如下特点：

1、C7609 CPU负载高达99%，从总部pingAA和BB的C3550都无法连通；

2、两矿用户反映可以访问其各自的内部网站；

3、A矿可以ping通网关X.X.60.1，但丢包严重；

4、B矿几乎无法ping通网关X.X.130.1；

5、总部用户也反映上网比平时明显要慢。

### 分析方案设计
 
用户基本网络拓扑如下图所示。

![UDP攻击案例图1](images/14/UDP攻击案例图1.png) 

A矿距集团总部大约15公里，之间采用100M光纤连接；B矿因距总部较远，因而对总部机关网络的访问是通过本矿一台C3550走2M通讯线路连接就近接入A的C3550，通过A网络实现与机关总部网络互联的。

A矿所属网络为VLAN 23，网关X.X.60.1指在C7609上，B矿所属网络为VLAN 22，网关X.X.130.1也指在C7609上，通过启用OSPF路由访问网络。

### 分析目标
初步判定怀疑有异常的网络行为导致路由器CPU负载增大，致使处理能力下降，从而影响网络性能。因此，分析出造成路由器C7609 CPU负载高的原因实际上也就能分析出网络访问慢的原因。

9.2.2　分析设备部署
因两个单位离机关总部很远，网络监控分析工作暂定在总部，将安装了科来网络分析系统的PC布设在了C7609，与其G2/42端口连接。为了进一步分析故障原因，在C7609上配置端口镜像，源端口为G4/5，目的端口为G2/42的镜像，通过连接在G2/42口的监控PC抓取数据包。

因C7609的G4/5端口到C3550的连接为百兆，在采用科来网络分析系统进行抓包时，“网络档案”采用100M方案，“本地子网”设置中添加X.X.60.0/23和X.X.130.0/23两个网段，数据包缓存设置为50M。此次抓包时间为2.11秒，数据包大小15.629MB，数据包文件名为dht。

同样，对于捕获的数据包分析，一般按照“我的图表”、“概要”、“诊断”的顺序，对数据包进行一个整体性的分析；按照“协议”、“IP端点”或“物理端点”、“IP/TCP/UDP会话”等内容对数据包做详尽的故障点分析；最后，结合“诊断”内容对造成故障的主机及故障原因做总结。
 
### 分析过程
 
#### 基本流量分析

通过“概要”模块查看基本流量信息，可以看到三个突出特点：1、带宽利用率高达66%；2、每秒数据包最大为13256；3、大包字节数为12.903MB，约占数据总量的83% （12.903MB / 15.629 MB），如下图所示。

![UDP攻击案例图2](images/14/UDP攻击案例图2.png) 

然后对A矿问题网段的基本流量进行分析，对该网段的概要分析，如下图所示。

![UDP攻击案例图3](images/14/UDP攻击案例图3.png) 

由图中的统计数据可以看出，A矿网络中发送数据量远远大于接收数据量，由于短时间内大量发包可能造成网络拥塞。按流量排序，该网段流量最高的内部主机为X.X.60.45，占总体流量为40%，且其发送数据包远远大约接收数据包，发送/接收比达到234，有明显的异常，如下图所示。

![UDP攻击案例图4](images/14/UDP攻击案例图4.png) 

#### 重点主机分析
X.X.60.45以2秒钟内发送了6千多个数据包，由于我们是在总部的路由器上抓到的数据，并不一定是全部数据，也就是说，该主机发送的数据包可能更多。其数据包解码，如下图所示。

![UDP攻击案例图5](images/14/UDP攻击案例图5.png) 

由上图可见，几乎所有数据包的源IP和源端口、目标IP和目标端口都相同，都是源为X.X.60.45:5444，目标为X.X.198.72:80之间的UDP通讯包，这些数据包之间间隔很短，大小完全相等，全部为1066字节，“Extra Data”数据项全部为填充块41。

可以初步判定这些数据包为伪造数据包，该主机通过高速、大量的伪造UDP大包向外网某一主机发起攻击，而此攻击大大消耗了核心交换机C7609的CPU资源并占用了A矿到机关总部的带宽资源。

#### 其他流量分析

进一步分析其他的主机流量。

因为C7609为核心交换机，以其为网关的直连网络多达六七十个，这些子网中的流量也会被监听抓取到，所以对除A、B两矿外的其它192网段及172网段的流量也需要做出分析，以判断是否存在可能的攻击。

对于172网段，按“字节”排序后，可以看到这段内的主机流量都很小，没有明显异常流量，将其排除在故障源之外。

对192网段按“字节”排序后，发现这段网络流量较高，大约7.952MB，占抓包文件的51%（7.952/15.629MB）。但仔细观察后发现，除主机X.X.43.176流量明显较高外，其他部分虽然流量较大，但发包的主机数目也多，各主机流量比较均衡，没有典型异常流量特征，属UDP下载包。

主机X.X.43.176流量明显高于其它主机，查看其通讯数据包，发现全部为UDP包，大小不等，分段长度随机，IP标识不同。虽然该主机流量明显高于其它主机，但其流量也应该为UDP下载包，如下图所示。

![UDP攻击案例图6](images/14/UDP攻击案例图6.png) 

通过分析并没有发现有其他的主机有明显异常流量。 

### 分析结论
 

经过分析，我们对本故障进行总结：A矿的主机X.X.60.45通过核心交换机C7609向外网主机X.X.198.72发送大量伪造的UDP大包，造成A到机关的100M线路阻塞，使得两矿用户访问总部及互联网的网络出现故障，同时由于大量的UDP攻击包造成C7609的CPU利用率高达99%，性能严重下降，影响了整个集团公司其它局域网络的路由转发及连接响应等服务，导致整个网络用户访问互联网慢。

在A矿将IP地址为X.X.60.45的用户强制下线后，C7609的CPU立刻下降到正常值范围内，大约为23%。B矿可以连通到机关总部并正常上互联网，此时，A矿X.X.60.2交换机可以ping通，其它上网功能恢复正常，总部用户上网速度也明显提高。

### 给我们的启示
 

本案例通过科来网络分析系统，对关键节点数据包级的网络流量，进行精细化分析与信息比对，在复杂的表象之下剥丝抽茧，快速定位问题主机与故障原因。再高级的攻击，都会产生流量，网络分析则是通过对流量的分析，发现的网络安全异常。

### 本节实验操作

1.打开udp_dns_flood.pcap样本数据文件，思考以下问题，并将分析过过程与答案写在实验报告中。

- 文件中，源IP端点有几个？目的端点有几个？使用什么统计工具分析最合适？

- 第3-第10分组，传递的数据包为什么类型？使用什么工具分析最合适？

- 请描述从样本数据中发现的事实，为什么会有这种情况发生？

2.打开udp_multicast_flood.pcap样本数据文件，思考以下问题，并将分析过过程与答案写在实验报告中。

- 文件中，源IP端点有几个？目的端点有几个？

- 对样本中的分组进行协议分级统计

- 为什么这些分组被wireshark着色为红色（TTL low or unexpected)? 这反映了什么问题？

- 请描述从样本数据中发现的事实，为什么会有这种情况发生？

## DDoS攻击缓解最佳实践

云计算环境下，主流的抗击DoS架构如下：

![阿里云ddos防护架构](images/14/阿里云ddos防护架构.png)

还有针对特殊行业的专用防护方案：

![阿里云游戏防护](images/14/阿里云游戏防护.jpg)

建议用户从以下几个方面着手缓解DDoS攻击的威胁：

### 缩小暴露面，隔离资源和不相关的业务，降低被攻击的风险。

#### 配置安全组

尽量避免将非业务必须的服务端口暴露在公网上，从而避免与业务无关的请求和访问。通过配置安全组可以有效防止系统被扫描或者意外暴露。

#### 使用专有网络（VPC）

通过专有网络VPC实现网络内部逻辑隔离，防止来自内网肉鸡的攻击。


### 优化业务架构，利用公共云的特性设计弹性伸缩和灾备切换的系统。

#### 科学评估业务架构性能

在业务部署前期或运营期间，技术团队应该对业务架构进行压力测试，以评估现有架构的业务吞吐处理能力，为DDoS防御提供详细的技术参数指导信息。

#### 弹性和冗余架构

通过负载均衡架构、或异地多中心的架构避免业务架构中出现单点故障。

如果业务在云上，可以灵活地使用负载均衡服务实现多台服务器的多点并发处理业务访问，将用户访问流量均衡分配到各个服务器上，降低单台服务器的压力，提升业务吞吐处理能力，这样可以有效缓解一定流量范围内的连接层DDoS攻击。

#### 部署弹性伸缩

弹性伸缩（Auto Scaling），是根据用户的业务需求和策略，经济地自动调整弹性计算资源的管理服务。通过部署弹性伸缩，系统可以有效的缓解会话层和应用层攻击，在遭受攻击时自动增加服务器，提升处理性能，避免业务遭受严重影响。


#### 优化DNS解析

通过智能解析的方式优化DNS解析，可以有效避免DNS流量攻击产生的风险。同时，建议您将业务托管至多家DNS服务商。

- 屏蔽未经请求发送的DNS响应信息
- 丢弃快速重传数据包
- 启用TTL
- 丢弃未知来源的DNS查询请求和响应数据
- 丢弃未经请求或突发的DNS请求
- 启动DNS客户端验证
- 对响应信息进行缓存处理
- 使用ACL的权限
- 利用ACL，BCP38及IP信誉功能

#### 提供余量带宽

通过服务器性能测试，评估正常业务环境下所能承受的带宽和请求数。在购买带宽时确保有一定的余量带宽，可以避免遭受攻击时带宽大于正常使用量而影响正常用户的情况。

### 服务器安全加固，提升服务器自身的连接数等性能。

对服务器上的操作系统、软件服务进行安全加固，减少可被攻击的点，增大攻击方的攻击成本：

- 确保服务器的系统文件是最新的版本，并及时更新系统补丁。
- 对所有服务器主机进行检查，清楚访问者的来源。
- 过滤不必要的服务和端口。例如，对于WWW服务器，只开放80端口，将其他所有端口关闭，或在防火墙上设置阻止策略。
- 限制同时打开的SYN半连接数目，缩短SYN半连接的timeout时间，限制SYN/ICMP流量。
- 仔细检查网络设备和服务器系统的日志。一旦出现漏洞或是时间变更，则说明服务器可能遭到了攻击。
- 限制在防火墙外进行网络文件共享。降低黑客截取系统文件的机会，若黑客以特洛伊木马替换它，文件传输功能将会陷入瘫痪。
- 充分利用网络设备保护网络资源。在配置路由器时应考虑针对流控、包过滤、半连接超时、垃圾包丢弃、来源伪造的数据包丢弃、SYN阀值、禁用ICMP和UDP广播的策略配置。
- 通过iptable之类的软件防火墙限制疑似恶意IP的TCP新建连接，限制疑似恶意IP的连接、传输速率。

### 做好业务监控和应急响应。

#### 关注基础DDoS防护监控

当业务遭受DDoS攻击时，基础DDoS默认会通过短信和邮件方式发出告警信息，针对大流量攻击基础DDoS防护也支持电话报警，建议您在接受到告警的第一时间进行应急处理。

关于配置告警消息接收人和语音告警方式，请查看DDoS基础防护消息接收人设置。

#### 云监控

云监控服务可用于收集、获取云资源的监控指标或用户自定义的监控指标，探测服务的可用性，并支持针对指标设置警报。

#### 建立应急响应预案

根据当前的技术业务架构和人员，提前准备应急技术预案，必要时，可以提前进行技术演练，以检验应急响应预案的合理性。

### 选择合适的商业安全方案。

云服务商既提供了免费的基础DDoS防护，也提供了DDoS防护包、高防IP、游戏盾等商业安全方案，用户也可以选择其他厂商的安全方案。

#### Web应用防火墙（WAF）

针对网站类应用，例如常见的http Flood(CC攻击)攻击，可以使用WAF可以提供针对连接层攻击、会话层攻击和应用层攻击进行有效防御。

#### DDoS防护包

DDoS防护包为云产品IP提供防御100G以内的DDoS攻击的防护能力，即时生效。

#### DDoS高级防护

针对大流量DDoS攻击，建议使用阿里云高防IP服务。

#### 游戏盾

游戏盾是针对游戏行业常见的DDoS攻击、CC攻击推出的行业解决方案。相比于高防IP服务，游戏盾解决方案的针对性更强，针对游戏行业的攻击防御效果更好、成本更低。


### 应当避免的事项

DDoS攻击是业内公认的行业公敌，DDoS攻击不仅影响被攻击者，同时也会对服务商网络的稳定性造成影响，从而对处于同一网络下的其他用户业务也会造成损失。

- 计算机网络是一个共享环境，需要多方共同维护稳定，部分行为可能会给整体网络和其他租户的网络带来影响，需要您注意：
- 避免使用阿里云产品机制搭建DDoS防护平台
- 避免释放处于黑洞状态的实例
- 避免为处于黑洞状态的服务器连续更换、解绑、增加SLB IP、弹性公网IP、NAT网关等IP类产品
- 避免通过搭建IP池进行防御，避免通过分摊攻击流量到大量IP上进行防御
- 避免利用阿里云非网络安全防御产品（包括但不限于CDN、OSS），前置自身有攻击的业务
- 避免使用多个账号的方式绕过上述规则


参考文档：https://help.aliyun.com/product/28396.html?spm=a2c4g.11186623.3.1.5ac83c37SSxq7E